In [3]:
!pip install pandas
!python -m spacy download pt_core_news_lg


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 8.1 MB/s eta 0:00:00a 0:00:04

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [8]:
import pandas as pd
import spacy

In [7]:
df = pd.read_csv("../../data/raw/patentes_inpi.csv")
df.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc
0,BR 11 2021 018393 0,02/03/2020,TRATAMENTO DE COLISÕES EM UPLINK,H04L 1/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,"A presente invenção se refere a métodos, sis...",H04L 1/18
1,BR 11 2021 018071 0,02/03/2020,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,H01T 13/14,https://busca.inpi.gov.br/pePI/servlet/Patente...,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,H01T 13/14 ; H01T 13/20 ; H01T 13/32 ; H0...
2,BR 11 2021 016947 4,02/03/2020,ANTICORPOS QUE RECONHECEM TAU,C07K 16/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,C07K 16/18 ; G01N 33/68
3,BR 10 2020 004169 0,02/03/2020,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,F24H 3/00,https://busca.inpi.gov.br/pePI/servlet/Patente...,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,F24H 3/008 ; F24H 4/06
4,BR 11 2021 006234 3,02/03/2020,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,C12N 15/10,https://busca.inpi.gov.br/pePI/servlet/Patente...,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,C12N 15/10


In [14]:
nlp = spacy.load("pt_core_news_lg")


def lemmatizer_spacy(text: str):
    """Lemmatizes the input text using spaCy."""
    str_text = str(text)
    doc = nlp(str_text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

In [15]:
df["titulo_lemmatized"] = df["titulo"].apply(lemmatizer_spacy)
df["resumo_lemmatized"] = df["resumo"].apply(lemmatizer_spacy)

In [20]:
# Compairing the original and lemmatized text
df[["titulo", "titulo_lemmatized"]].head(10)

,titulo,titulo_lemmatized
0,TRATAMENTO DE COLISÕES EM UPLINK,tratamento colisões uplink
1,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,alojamento vela ignição proteção anticorrosiva...
2,ANTICORPOS QUE RECONHECEM TAU,anticorpos reconhecem tau
3,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,aquecedor ar lenha dupla exaustão utilizar amb...
4,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,bibliotecas células únicas núcleos únicos alto...
5,CAPACIDADE DE MONITORAMENTO DE CANAL DE CONTRO...,capacidade monitoramento canal controle comuni...
6,"CÉLULA, CONSTRUÇÃO DE EXPRESSÃO, PLANTA OU PAR...",célula construção expressão planta planta conj...
7,COMPARTILHAMENTO DE TEMPO DE OCUPAÇÃO DE CANAL...,compartilhamento ocupação canal cot uplink ul ...
8,COMPOSIÇÃO FARMACÊUTICA BASEADA EM COMPOSTOS 1...,"composição farmacêutica baseada compostos 1,3,..."
9,CONFIGURAÇÃO DE TURBOMÁQUINA COMPRESSOR-EXPANS...,configuração turbomáquina compressor-expansor ...


In [21]:
# Compairing the original and lemmatized text
df[["resumo", "resumo_lemmatized"]].head(10)

,resumo,resumo_lemmatized
0,"A presente invenção se refere a métodos, sis...",presente invenção referir método sistema di...
1,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,alojamento vela ignição proteção anticorros...
2,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,anticorpo reconhecer tau invenção fornecer ...
3,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,aquecedor ar lenha dupla exaustao utilizar ...
4,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,biblioteca célula único núcleo único alto r...
5,CAPACIDADE DE MONITORAMENTO DE CANAL DE CONT...,capacidade monitoramento canal controle com...
6,"CÉLULA, CONSTRUÇÃO DE EXPRESSÃO, PLANTA OU P...",célula construção expressão planta planta c...
7,COMPARTILHAMENTO DE TEMPO DE OCUPAÇÃO DE CAN...,compartilhamento ocupação canal cot uplink ...
8,COMPOSIÇÃO FARMACÊUTICA BASEADA EM COMPOSTOS...,"composição farmacêutico basear composto 1,3..."
9,CONFIGURAÇÃO DE TURBOMÁQUINA COMPRESSOREXPAN...,configuração turbomáquina compressorexpanso...


In [91]:
df.to_csv("../../data/processed/patentes_inpi_lemmatized.csv", index=False)

In [22]:
# Importar bibliotecas para clusterização
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [92]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def optimal_kmeans(texts, nlp, max_clusters=10):
    """
    Realiza KMeans nos textos após calcular a quantidade ideal de clusters.
    
    Args:
        texts (list): Lista de textos para clusterização.
        nlp (spacy.lang): Modelo spaCy para gerar vetores.
        max_clusters (int): Número máximo de clusters a testar.
    
    Returns:
        dict: Contém o melhor número de clusters, o modelo KMeans treinado,
              o silhouette score e os rótulos dos clusters.
    """
    # Função para gerar vetores do spaCy
    def get_spacy_vectors(texts, nlp):
        return [nlp(str(text)).vector for text in texts]
    
    # Gerar vetores para os textos
    vectors = get_spacy_vectors(texts, nlp)
    
    best_k = 2
    best_score = -1
    best_model = None
    best_labels = None
    
    # Testar diferentes números de clusters
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(vectors)
        score = silhouette_score(vectors, labels)
        
        print(f"Clusters: {k}, Silhouette Score: {score}")
        
        if score > best_score:
            best_k = k
            best_score = score
            best_model = kmeans
            best_labels = labels
    
    return {
        "best_k": best_k,
        "best_score": best_score,
        "model": best_model,
        "labels": best_labels.tolist()
    }

In [94]:
# Aplicar a função para as colunas 'resumo' e 'resumo_lemmatized'
print("Analisando 'resumo':")
result_resumo = optimal_kmeans(df['resumo'], nlp)

print("\nAnalisando 'resumo_lemmatized':")
result_resumo_lemmatized = optimal_kmeans(df['resumo_lemmatized'], nlp)

# Adicionar os clusters ao DataFrame
df['cluster_resumo'] = result_resumo['labels']
df['cluster_resumo_lemmatized'] = result_resumo_lemmatized['labels']

# Exibir os melhores resultados
print(f"\nMelhor número de clusters para 'resumo': {result_resumo['best_k']}")
print(f"Silhouette Score para 'resumo': {result_resumo['best_score']}")

print(f"\nMelhor número de clusters para 'resumo_lemmatized': {result_resumo_lemmatized['best_k']}")
print(f"Silhouette Score para 'resumo_lemmatized': {result_resumo_lemmatized['best_score']}")

Analisando 'resumo':
Clusters: 2, Silhouette Score: 0.6200077533721924
Clusters: 2, Silhouette Score: 0.6200077533721924
Clusters: 3, Silhouette Score: 0.27617594599723816
Clusters: 3, Silhouette Score: 0.27617594599723816
Clusters: 4, Silhouette Score: 0.19419431686401367
Clusters: 4, Silhouette Score: 0.19419431686401367
Clusters: 5, Silhouette Score: 0.19265006482601166
Clusters: 5, Silhouette Score: 0.19265006482601166
Clusters: 6, Silhouette Score: 0.16517505049705505
Clusters: 6, Silhouette Score: 0.16517505049705505
Clusters: 7, Silhouette Score: 0.14385341107845306
Clusters: 7, Silhouette Score: 0.14385341107845306
Clusters: 8, Silhouette Score: 0.1351085901260376
Clusters: 9, Silhouette Score: 0.1288367360830307
Clusters: 8, Silhouette Score: 0.1351085901260376
Clusters: 9, Silhouette Score: 0.1288367360830307
Clusters: 10, Silhouette Score: 0.12637899816036224

Analisando 'resumo_lemmatized':
Clusters: 10, Silhouette Score: 0.12637899816036224

Analisando 'resumo_lemmatized':